In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import gamma
from time import time
from copy import deepcopy

In [2]:
base_dpe_df=pd.read_csv('base_logement_agregee.csv',sep=';',decimal='.')

# Filtre résidences principales
base_dpe_df=base_dpe_df[(base_dpe_df.occupancy_status!="non primary residence")]

# Simplification : low rent housing = renter et free accomodation = owner
base_dpe_df.loc[(base_dpe_df.occupancy_status=="low rent housing"),"occupancy_status"]="renter"
base_dpe_df.loc[(base_dpe_df.occupancy_status=="free accomodation"),"occupancy_status"]="owner"

# Suppression des doublons
base_dpe_df["surface"]=base_dpe_df["surface"]*base_dpe_df["IPONDL"]
base_dpe_df["energy_consumption"]=base_dpe_df["surface"]*base_dpe_df["energy_consumption"]
base_dpe_df=base_dpe_df.groupby(["construction_year_class","heating_system","living_area_class",\
                                     "residential_type","occupancy_status","heating_fuel","energy_class"]).sum()
base_dpe_df["energy_consumption"]=base_dpe_df["energy_consumption"]/base_dpe_df["surface"]
base_dpe_df["surface"]=base_dpe_df["surface"]/base_dpe_df["IPONDL"]

# Réindexation
base_dpe_df=base_dpe_df.reset_index()

# Affichage
base_dpe_df

,construction_year_class,heating_system,living_area_class,residential_type,occupancy_status,heating_fuel,energy_class,IPONDL,energy_consumption,surface
0,"[1000, 1918]",Autres,120 m² ou plus,apartment,owner,other,A,13,36.685197,165.965968
1,"[1000, 1918]",Autres,120 m² ou plus,apartment,owner,other,B,68,74.694238,165.965968
2,"[1000, 1918]",Autres,120 m² ou plus,apartment,owner,other,C,228,121.959658,165.965968
3,"[1000, 1918]",Autres,120 m² ou plus,apartment,owner,other,D,330,186.160758,165.965968
4,"[1000, 1918]",Autres,120 m² ou plus,apartment,owner,other,E,257,271.905932,165.965968
...,...,...,...,...,...,...,...,...,...,...
8619,"[2013, 2100]",Pompes à chaleur électricité,Moins de 30 m²,house,renter,electricity,B,39,61.113090,24.463885
8620,"[2013, 2100]",Pompes à chaleur électricité,Moins de 30 m²,house,renter,electricity,C,5,114.550832,24.463885
8621,"[2013, 2100]",Pompes à chaleur électricité,Moins de 30 m²,house,renter,electricity,D,34,194.881128,24.463885
8622,"[2013, 2100]",Pompes à chaleur électricité,Moins de 30 m²,house,renter,electricity,E,14,265.403257,24.463885


In [3]:
# Classes énergies moyennes
copy_dpe_df=base_dpe_df.copy(deep=True)
copy_dpe_df["surface"]=copy_dpe_df["surface"]*copy_dpe_df["IPONDL"]
copy_dpe_df["energy_consumption"]=copy_dpe_df["surface"]*copy_dpe_df["energy_consumption"]
energy_class_df=copy_dpe_df.groupby(["energy_class"]).sum()
energy_class_df["energy_consumption"]=energy_class_df["energy_consumption"]/energy_class_df["surface"]
energy_class_df["surface"]=energy_class_df["surface"]/energy_class_df["IPONDL"]
print(energy_class_df)

               IPONDL  energy_consumption    surface
energy_class                                        
A              551102           41.024614  96.664379
B             1163714           69.283210  89.787852
C             3515951          124.969959  97.371169
D             7917477          190.574440  91.232118
E             8488730          275.640812  86.114193
F             4818397          380.040701  85.079106
G             2287167          545.867603  84.245531


In [4]:
# Performances appartements maisons
copy33_dpe_df=base_dpe_df.copy(deep=True)
copy33_dpe_df["surface"]=copy33_dpe_df["surface"]*copy33_dpe_df["IPONDL"]
copy33_dpe_df["energy_consumption"]=copy33_dpe_df["energy_consumption"]*copy33_dpe_df["surface"]
#copy33_dpe_df.groupby(["residential_type"]).sum()["energy_consumption"]/copy33_dpe_df.groupby(["residential_type"]).sum()["surface"]
copy33_dpe_df=copy33_dpe_df.groupby(["residential_type","energy_class"]).sum()
copy33_dpe_df["energy_consumption"]=copy33_dpe_df["energy_consumption"]/copy33_dpe_df["surface"]
copy33_dpe_df

IPONDL  energy_consumption       surface
residential_type energy_class                                           
apartment        A              222713           40.927693  1.354297e+07
                 B              610742           69.268233  3.881071e+07
                 C             1651680          124.573855  1.131101e+08
                 D             3639989          190.516920  2.308167e+08
                 E             3692324          274.595846  2.180681e+08
                 F             1892982          377.560167  1.062962e+08
                 G              753012          542.472589  3.889192e+07
house            A              328389           41.057653  3.972897e+07
                 B              552972           69.292061  6.567667e+07
                 C             1864271          125.165400  2.292421e+08
                 D             4277488          190.601452  4.915115e+08
                 E             4796406          276.085070  5.129320e+08
                 F             2925415          380.909043  3.036487e+08
                 G             1534155          546.726158  1.537917e+08

In [5]:
# Années de construction
copy_dpe_df=base_dpe_df.copy(deep=True)[base_dpe_df.residential_type=="house"]
copy_dpe_df["surface"]=copy_dpe_df["surface"]*copy_dpe_df["IPONDL"]
copy_dpe_df["energy_consumption"]=copy_dpe_df["surface"]*copy_dpe_df["energy_consumption"]
energy_class_df=copy_dpe_df.groupby(["construction_year_class"]).sum()
energy_class_df["energy_consumption"]=energy_class_df["energy_consumption"]/energy_class_df["surface"]
energy_class_df["surface"]=energy_class_df["surface"]/energy_class_df["IPONDL"]
print(energy_class_df)

                          IPONDL  energy_consumption     surface
construction_year_class                                         
[1000, 1918]             2538405          327.017462  113.439113
[1919, 1945]             1609627          323.413234  101.461649
[1946, 1970]             2715996          316.258805  100.127196
[1971, 1990]             4681958          269.016198  109.877359
[1991, 2005]             2613593          199.996783  118.142405
[2006, 2012]             1367727          189.667862  119.453359
[2013, 2100]              751790           66.775575  115.351148


In [6]:
# Performances appartements/maisons neufs
copy2_dpe_df=base_dpe_df.copy(deep=True)[base_dpe_df.construction_year_class=="[2013, 2100]"]
copy2_dpe_df["surface"]=copy2_dpe_df["surface"]*copy2_dpe_df["IPONDL"]
copy2_dpe_df["energy_consumption"]=copy2_dpe_df["surface"]*copy2_dpe_df["energy_consumption"]
energy2_class_df=copy2_dpe_df.groupby(["residential_type","energy_class"]).sum()
energy2_class_df["energy_consumption"]=energy2_class_df["energy_consumption"]/energy2_class_df["surface"]
energy2_class_df["surface"]=energy2_class_df["surface"]/energy2_class_df["IPONDL"]
energy2_class_df

IPONDL  energy_consumption     surface
residential_type energy_class                                        
apartment        A             199940           41.288619   60.155177
                 B             389440           64.495932   58.866875
                 C              72582          115.592125   55.658032
                 D              50216          190.138676   51.678812
                 E              23207          268.784542   48.005318
                 F               4439          380.162905   44.301593
                 G                691          560.562114   43.056572
house            A             300166           41.416793  121.053925
                 B             347283           64.227604  113.357891
                 C              52165          115.607617  109.940285
                 D              39258          188.529224  105.548862
                 E              11495          264.284034   90.104655
                 F               1258          370.407109   70.963210
                 G                165          504.648946   76.376024

In [7]:
# Types de chauffage
copy3_dpe_df=base_dpe_df.copy(deep=True)
copy3_dpe_df["surface"]=copy3_dpe_df["surface"]*copy3_dpe_df["IPONDL"]
copy3_dpe_df["energy_consumption"]=copy3_dpe_df["surface"]*copy3_dpe_df["energy_consumption"]
energy3_class_df=copy3_dpe_df.groupby(["residential_type","heating_system","heating_fuel"]).sum()
energy3_class_df["energy_consumption"]=energy3_class_df["energy_consumption"]/energy3_class_df["surface"]
energy3_class_df["surface"]=energy3_class_df["surface"]/energy3_class_df["IPONDL"]
energy3_class_df

IPONDL  \
residential_type heating_system               heating_fuel            
apartment        Autres                       other          103031   
                 Chaudière - autres           other          217456   
                 Chaudière fioul              oil            475230   
                 Chaudière gaz                gas           5334267   
                 Chauffage urbain             other         1554877   
                 Chauffage électrique         electricity   3995209   
                 Pompes à chaleur électricité electricity    783372   
house            Autres                       other         2978761   
                 Chaudière - autres           other          796863   
                 Chaudière fioul              oil           2428267   
                 Chaudière gaz                gas           5168802   
                 Chauffage urbain             other           13311   
                 Chauffage électrique         electricity   4442744   
                 Pompes à chaleur électricité electricity    450348   

                                                            energy_consumption  \
residential_type heating_system               heating_fuel                       
apartment        Autres                       other                 259.613225   
                 Chaudière - autres           other                 222.830271   
                 Chaudière fioul              oil                   272.792252   
                 Chaudière gaz                gas                   208.241804   
                 Chauffage urbain             other                 247.289805   
                 Chauffage électrique         electricity           279.481761   
                 Pompes à chaleur électricité electricity           270.953640   
house            Autres                       other                 270.918417   
                 Chaudière - autres           other                 246.582400   
                 Chaudière fioul              oil                   288.880370   
                 Chaudière gaz                gas                   227.689385   
                 Chauffage urbain             other                 280.796916   
                 Chauffage électrique         electricity           281.771239   
                 Pompes à chaleur électricité electricity           269.175392   

                                                               surface  
residential_type heating_system               heating_fuel              
apartment        Autres                       other          70.110694  
                 Chaudière - autres           other          65.465799  
                 Chaudière fioul              oil            68.656689  
                 Chaudière gaz                gas            66.283436  
                 Chauffage urbain             other          62.634561  
                 Chauffage électrique         electricity    52.973740  
                 Pompes à chaleur électricité electricity    54.694050  
house            Autres                       other         108.482832  
                 Chaudière - autres           other         129.069934  
                 Chaudière fioul              oil           116.852067  
                 Chaudière gaz                gas           109.250176  
                 Chauffage urbain             other          86.516098  
                 Chauffage électrique         electricity   105.717031  
                 Pompes à chaleur électricité electricity   113.845955

In [8]:
# Correction des modes de chauffage
for i in base_dpe_df.index:
    if base_dpe_df.loc[i,"heating_system"]=="Pompes à chaleur électricité":
        base_dpe_df.loc[i,"heating_system"]="Pompes à chaleur air-eau"
    if base_dpe_df.loc[i,"residential_type"]=="apartment" and base_dpe_df.loc[i,"heating_system"] in ["Autres","Chaudière - autres"]:
        base_dpe_df.loc[i,"heating_system"]="Chaudière gaz"
    if base_dpe_df.loc[i,"residential_type"]=="house":
        if base_dpe_df.loc[i,"heating_system"]=="Chaudière - autres":
            base_dpe_df.loc[i,"heating_system"]="Chaudière fioul"
        elif base_dpe_df.loc[i,"heating_system"]=="Autres":
            base_dpe_df.loc[i,"heating_system"]="Biomasse"
            index=list(base_dpe_df.loc[i][["construction_year_class","heating_system",
                                      "living_area_class","residential_type",
                                      "occupancy_status","heating_fuel","energy_class"]])
            IPONDL=base_dpe_df.loc[i,"IPONDL"]
            energy=base_dpe_df.loc[i,"energy_consumption"]
            surface=base_dpe_df.loc[i,"surface"]
            IPONDL_gaz=round(IPONDL/10)
            IPONDL_elec=round(IPONDL/3)
            IPONDL_PAC=round(IPONDL/6)
            base_dpe_df.loc[i,"IPONDL"]=IPONDL-IPONDL_gaz-IPONDL_elec-IPONDL_PAC
            df_append=pd.DataFrame([[index[0]]+["Chaudière gaz"]+index[2::]
                                +[IPONDL_gaz,energy,surface],
                                    [index[0]]+["Chauffage électrique"]+index[2::]
                                +[IPONDL_elec,energy,surface],
                                    [index[0]]+["Pompes à chaleur air-eau"]+index[2::]
                                +[IPONDL_PAC,energy,surface]],
                                columns=["construction_year_class","heating_system",
                                      "living_area_class","residential_type",
                                      "occupancy_status","heating_fuel","energy_class",
                                      "IPONDL","energy_consumption","surface"])
            base_dpe_df=pd.concat([base_dpe_df,df_append],ignore_index=True)

# Suppression des doublons créés
base_dpe_df["surface"]=base_dpe_df["surface"]*base_dpe_df["IPONDL"]
base_dpe_df["energy_consumption"]=base_dpe_df["surface"]*base_dpe_df["energy_consumption"]
base_dpe_df=base_dpe_df.groupby(["construction_year_class","heating_system","living_area_class",\
                                     "residential_type","occupancy_status","energy_class"]).sum()
base_dpe_df["energy_consumption"]=base_dpe_df["energy_consumption"]/base_dpe_df["surface"]
base_dpe_df["surface"]=base_dpe_df["surface"]/base_dpe_df["IPONDL"]

# Réindexation
base_dpe_df=base_dpe_df.reset_index()

# Suppression des nan
base_dpe_df=base_dpe_df.dropna()

# Affichage
base_dpe_df

,construction_year_class,heating_system,living_area_class,residential_type,occupancy_status,energy_class,IPONDL,energy_consumption,surface
0,"[1000, 1918]",Biomasse,120 m² ou plus,house,owner,A,262,38.190806,169.088298
1,"[1000, 1918]",Biomasse,120 m² ou plus,house,owner,B,1300,72.855568,169.088298
2,"[1000, 1918]",Biomasse,120 m² ou plus,house,owner,C,6537,124.862578,169.088298
3,"[1000, 1918]",Biomasse,120 m² ou plus,house,owner,D,13941,191.462752,169.088298
4,"[1000, 1918]",Biomasse,120 m² ou plus,house,owner,E,15922,277.163129,169.088298
...,...,...,...,...,...,...,...,...,...
6940,"[2013, 2100]",Pompes à chaleur air-eau,Moins de 30 m²,house,renter,B,43,62.424470,24.463885
6941,"[2013, 2100]",Pompes à chaleur air-eau,Moins de 30 m²,house,renter,C,9,111.314530,24.463885
6942,"[2013, 2100]",Pompes à chaleur air-eau,Moins de 30 m²,house,renter,D,35,195.161511,24.463885
6943,"[2013, 2100]",Pompes à chaleur air-eau,Moins de 30 m²,house,renter,E,14,265.403257,24.463885


In [9]:
# Modes de chauffage corrigés
copy3_dpe_df=base_dpe_df.copy(deep=True)
copy3_dpe_df["surface"]=copy3_dpe_df["surface"]*copy3_dpe_df["IPONDL"]
copy3_dpe_df["energy_consumption"]=copy3_dpe_df["surface"]*copy3_dpe_df["energy_consumption"]
energy3_class_df=copy3_dpe_df.groupby(["heating_system"]).sum()
energy3_class_df["energy_consumption"]=energy3_class_df["energy_consumption"]/energy3_class_df["surface"]
energy3_class_df["surface"]=energy3_class_df["surface"]/energy3_class_df["IPONDL"]
energy3_class_df

,IPONDL,energy_consumption,surface
heating_system,,,
Biomasse,1191523,270.919827,108.481893
Chaudière fioul,3700360,277.251081,113.293515
Chaudière gaz,11121418,222.218583,87.402463
Chauffage urbain,1568188,247.681393,62.837271
Chauffage électrique,9430867,279.675571,83.664524
Pompes à chaleur air-eau,1730182,270.323457,85.525088


In [10]:
# Correction énergie finale

C_EP=2.58# Coefficient énergie primaire électricité
taux_ECS=0.1356# Part de l'ECS dans la consommation finale d'énergie des logements
taux_clim=0.0034# Part de la clim dans la consommation finale d'énergie des logements
taux_heat=1-taux_ECS-taux_clim# Part du chauffage dans la consommation finale d'énergie des logements

# Conversion de l'énergie primaire en énergie finale
def energy_to_class(energy):
    if energy<=50:
        return "A"
    elif energy<=90:
        return "B"
    elif energy<=150:
        return "C"
    elif energy<=230:
        return "D"
    elif energy<=330:
        return "E"
    elif energy<=450:
        return "F"
    else:
        return "G"

index1=(base_dpe_df.heating_system=="Biomasse")|(base_dpe_df.heating_system=="Chaudière fioul")|\
(base_dpe_df.heating_system=="Chauffage urbain")
base_dpe_df.loc[index1,"energy_consumption"]=base_dpe_df.loc[index1,"energy_consumption"]/(taux_heat+C_EP*(taux_clim+taux_ECS))

index2=(base_dpe_df.heating_system=="Chaudière gaz")
base_dpe_df.loc[index2,"energy_consumption"]=base_dpe_df.loc[index2,"energy_consumption"]/(taux_heat+C_EP*taux_clim+taux_ECS)

index3=(base_dpe_df.heating_system=="Chauffage électrique")|(base_dpe_df.heating_system=="Pompes à chaleur air-eau")
base_dpe_df.loc[index3,"energy_consumption"]=base_dpe_df.loc[index3,"energy_consumption"]/C_EP

base_dpe_df["energy_class"]=base_dpe_df["energy_consumption"].apply(lambda x: energy_to_class(x))

In [11]:
# Energie par mode de chauffage
copy3_dpe_df=base_dpe_df.copy(deep=True)
copy3_dpe_df["surface"]=copy3_dpe_df["surface"]*copy3_dpe_df["IPONDL"]
copy3_dpe_df["energy_consumption"]=copy3_dpe_df["surface"]*copy3_dpe_df["energy_consumption"]
energy3_class_df=copy3_dpe_df.groupby(["heating_system"]).sum()
energy3_class_df["energy_consumption"]=energy3_class_df["energy_consumption"]/energy3_class_df["surface"]
energy3_class_df["surface"]=energy3_class_df["surface"]/energy3_class_df["IPONDL"]
energy3_class_df

,IPONDL,energy_consumption,surface
heating_system,,,
Biomasse,1191523,222.134622,108.481893
Chaudière fioul,3700360,227.325791,113.293515
Chaudière gaz,11121418,221.031203,87.402463
Chauffage urbain,1568188,203.080790,62.837271
Chauffage électrique,9430867,108.401384,83.664524
Pompes à chaleur air-eau,1730182,104.776534,85.525088


In [12]:
# Calcul de la consommation réelle, ajustement des coefficients de correction
copy32_dpe_df=base_dpe_df.copy(deep=True)
d_corr_class={"A":1.3,"B":1.1,"C":0.85,"D":0.75,"E":0.65,"F":0.6,"G":0.55}
copy32_dpe_df["energy_consumption"]=copy32_dpe_df["energy_consumption"]*copy32_dpe_df["energy_class"].apply(lambda x: d_corr_class[x])
copy32_dpe_df["surface"]=copy32_dpe_df["surface"]*copy32_dpe_df["IPONDL"]
copy32_dpe_df["energy_consumption"]=copy32_dpe_df["energy_consumption"]*copy32_dpe_df["surface"]
print(copy32_dpe_df["energy_consumption"].sum()/copy32_dpe_df["surface"].sum())
print(copy32_dpe_df.loc[(copy32_dpe_df.heating_system=="Chauffage électrique"),"energy_consumption"].sum()
      +copy32_dpe_df.loc[(copy32_dpe_df.heating_system=="Pompes à chaleur air-eau"),"energy_consumption"].sum()/2.8)
print(18.8*(copy32_dpe_df.loc[(copy32_dpe_df.heating_system=="Chauffage électrique"),"surface"].sum()
      +copy32_dpe_df.loc[(copy32_dpe_df.heating_system=="Pompes à chaleur air-eau"),"surface"].sum()/2.8))

132.36495092999667
80731868248.01009
15827284703.00388


In [13]:
# Performances appartements maisons
copy33_dpe_df=base_dpe_df.copy(deep=True)
copy33_dpe_df["surface"]=copy33_dpe_df["surface"]*copy33_dpe_df["IPONDL"]
copy33_dpe_df["energy_consumption"]=copy33_dpe_df["energy_consumption"]*copy33_dpe_df["surface"]
#copy33_dpe_df.groupby(["residential_type"]).sum()["energy_consumption"]/copy33_dpe_df.groupby(["residential_type"]).sum()["surface"]
copy33_dpe_df=copy33_dpe_df.groupby(["residential_type","energy_class"]).sum()
copy33_dpe_df["energy_consumption"]=copy33_dpe_df["energy_consumption"]/copy33_dpe_df["surface"]
copy33_dpe_df

IPONDL  energy_consumption       surface
residential_type energy_class                                           
apartment        A              436668           36.604719  2.805986e+07
                 B             1913782           70.616466  1.127633e+08
                 C             3910283          119.874078  2.263728e+08
                 D             3517717          191.087679  2.218794e+08
                 E             1831478          277.676663  1.166567e+08
                 F              677263          381.747799  4.339448e+07
                 G              176251          503.640311  1.041031e+07
house            A              802965           33.667345  9.893473e+07
                 B             1976533           71.797415  2.251596e+08
                 C             4490942          119.133630  5.042538e+08
                 D             4743737          189.558372  5.240979e+08
                 E             2735482          281.381241  2.851805e+08
                 F             1041661          398.034114  1.124773e+08
                 G              487776          500.876203  4.642786e+07

In [12]:
# Répartitions des surfaces et statuts d'occupation appartements/maisons
copy4_dpe_df=base_dpe_df.copy(deep=True)
copy4_dpe_df["surface"]=copy4_dpe_df["surface"]*copy3_dpe_df["IPONDL"]
copy4_dpe_df["energy_consumption"]=copy4_dpe_df["surface"]*copy4_dpe_df["energy_consumption"]
energy4_class_df=copy4_dpe_df.groupby(["residential_type","living_area_class","occupancy_status"]).sum()
energy4_class_df["energy_consumption"]=energy4_class_df["energy_consumption"]/energy4_class_df["surface"]
energy4_class_df["surface"]=energy4_class_df["surface"]/energy4_class_df["IPONDL"]
energy4_class_df

IPONDL  \
residential_type living_area_class occupancy_status            
apartment        120 m² ou plus    owner              178868   
                                   renter              70986   
                 De 100 à 120 m²   owner              270058   
                                   renter             192330   
                 De 30 à 40 m²     owner              231851   
                                   renter            1224795   
                 De 40 à 60 m²     owner              803194   
                                   renter            2694518   
                 De 60 à 80 m²     owner             1256664   
                                   renter            2654631   
                 De 80 à 100 m²    owner              747460   
                                   renter             920029   
                 Moins de 30 m²    owner              123813   
                                   renter            1094245   
house            120 m² ou plus    owner             4052032   
                                   renter             209591   
                 De 100 à 120 m²   owner             3454530   
                                   renter             357626   
                 De 30 à 40 m²     owner               84020   
                                   renter              79604   
                 De 40 à 60 m²     owner              441321   
                                   renter             317729   
                 De 60 à 80 m²     owner             1597618   
                                   renter             745916   
                 De 80 à 100 m²    owner             4023936   
                                   renter             863072   
                 Moins de 30 m²    owner               24301   
                                   renter              27800   

                                                     energy_consumption  \
residential_type living_area_class occupancy_status                       
apartment        120 m² ou plus    owner                     193.471510   
                                   renter                    187.608199   
                 De 100 à 120 m²   owner                     177.803099   
                                   renter                    174.951425   
                 De 30 à 40 m²     owner                     172.364755   
                                   renter                    167.436928   
                 De 40 à 60 m²     owner                     185.651712   
                                   renter                    180.270616   
                 De 60 à 80 m²     owner                     173.471303   
                                   renter                    168.764724   
                 De 80 à 100 m²    owner                     177.279759   
                                   renter                    174.158865   
                 Moins de 30 m²    owner                     168.416139   
                                   renter                    153.101427   
house            120 m² ou plus    owner                     173.597013   
                                   renter                    179.827488   
                 De 100 à 120 m²   owner                     171.140750   
                                   renter                    164.291365   
                 De 30 à 40 m²     owner                     219.150455   
                                   renter                    194.148721   
                 De 40 à 60 m²     owner                     234.485675   
                                   renter                    204.353822   
                 De 60 à 80 m²     owner                     214.406575   
                                   renter                    181.708386   
                 De 80 à 100 m²    owner                     198.682127   
                                   renter                    173.925125   
                 Moins de 30 m²  

In [13]:
# Certains traitements sont plus rapides avec les listes
L_dpe_index=[]
L_dpe_var=[]
for i in base_dpe_df.index:
    row=list(base_dpe_df.loc[i])
    L_dpe_index.append(row[0:-3])
    L_dpe_var.append(row[-3::])

In [14]:
## Calcul transitions chauffage
d_heat={"Biomasse":0,
"Chaudière fioul":1,
"Chaudière gaz":2,
"Chauffage urbain":3,
"Chauffage électrique":4,
"Pompes à chaleur air-eau":5,
"Pompes à chaleur air-air":6,
"Pompes à chaleur hybride":7}

L_heat=["Biomasse","Chaudière fioul","Chaudière gaz",
"Chauffage urbain","Chauffage électrique","Pompes à chaleur air-eau",
"Pompes à chaleur air-air","Pompes à chaleur hybride"]

L_trans_house=[[1,0,0,0,0,0,0,0],
[0.15,0,0,0,0,0.8,0.05,0],
[0.15,0,0.05,0,0,0.6,0.05,0.15],
[0,0,0,1,0,0,0,0],
[0.1,0,0,0,0.2,0.05,0.65,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]]

L_trans_apartment=[[1,0,0,0,0,0,0,0],
[0,0,0,0.2,0,0.8,0,0],
[0,0,0.44,0.07,0.11,0.28,0.06,0.04],
[0,0,0,1,0,0,0,0],
[0,0,0,0,0.65,0,0.35,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]]

L_heat_house=[0 for i in range(8)]
L_heat_apartment=[0 for i in range(8)]
for i in range(len(L_dpe_index)):
    if L_dpe_index[i][3]=="house":
        L_heat_house[d_heat[L_dpe_index[i][1]]]+=L_dpe_var[i][0]
    else:
        L_heat_apartment[d_heat[L_dpe_index[i][1]]]+=L_dpe_var[i][0]
        
N_trans_house=[[L_heat_house[i]*L_trans_house[i][j] for j in range(8)] for i in range(8)]
N_trans_apartment=[[L_heat_apartment[i]*L_trans_apartment[i][j] for j in range(8)] for i in range(8)]

In [15]:
d_surfaces={"Moins de 30 m²":0,
"De 30 à 40 m²":1,
"De 40 à 60 m²":2,
"De 60 à 80 m²":3,
"De 80 à 100 m²":4,
"De 100 à 120 m²":5,
"120 m² ou plus":6}
L_surfaces=["Moins de 30 m²",
"De 30 à 40 m²",
"De 40 à 60 m²",
"De 60 à 80 m²",
"De 80 à 100 m²",
"De 100 à 120 m²",
"120 m² ou plus"]
d_status={"owner":0,"renter":1}
L_status=["owner","renter"]
L_taux_new_house=[[0,0] for i in range(7)]
L_taux_new_apartment=[[0,0] for i in range(7)]
L_surf_house=[0 for i in range(7)]
L_surf_apartment=[0 for i in range(7)]
for i in range(len(L_dpe_index)):
    if L_dpe_index[i][3]=="house":
        L_taux_new_house[d_surfaces[L_dpe_index[i][2]]][d_status[L_dpe_index[i][4]]]+=L_dpe_var[i][0]
        L_surf_house[d_surfaces[L_dpe_index[i][2]]]=L_dpe_var[i][2]
    else:
        L_taux_new_apartment[d_surfaces[L_dpe_index[i][2]]][d_status[L_dpe_index[i][4]]]+=L_dpe_var[i][0]
        L_surf_apartment[d_surfaces[L_dpe_index[i][2]]]=L_dpe_var[i][2]
s_house=sum([sum(x) for x in L_taux_new_house])
s_apartment=sum([sum(x) for x in L_taux_new_apartment])
for i in range(7):
    for j in range(2):
        L_taux_new_house[i][j]=L_taux_new_house[i][j]/s_house
        L_taux_new_apartment[i][j]=L_taux_new_apartment[i][j]/s_apartment

In [16]:
### Modèle complet efficace (listes)

## Fin de vie des logements
duree_de_vie=100
annee_ref=2020

# Les plus vieux (avant 1918)
#tau=1/duree_de_vie# Taux de démolition
#index_old=(base_dpe_df.construction_year_class=="[1000, 1918]")
#base_dpe_df.loc[index_old,"IPONDL"]=np.random.binomial(base_dpe_df.loc[index_old,"IPONDL"],1-tau)

# Après 1918 : application d'une loi de Weibull de paramètre k=1.1
# et mu=duree_de_vie (lambda=duree_de_vie/Gamma(1+1/k))
d_const_year={'[1000, 1918]':1900,'[1919, 1945]':1932, '[1946, 1970]':1958, '[1971, 1990]':1980,
       '[1991, 2005]':1998, '[2006, 2012]':2009, '[2013, 2100]':2016}

k=2.2# Paramètre choisi pour égaliser le nombre de logements déclassés par RTE en 2035 (2,5 M entre 2018 et 2035)
lamb=duree_de_vie/gamma(1+1/k)

def survie(IPONDL,construction_year_class,annee_en_cours):
    if construction_year_class=='[1000, 1918]':
        return IPONDL
    else:
        return np.random.binomial(IPONDL,np.exp(-((annee_en_cours-d_const_year[construction_year_class])/lamb)**k+((annee_en_cours-1-d_const_year[construction_year_class])/lamb)**k))

def survie_global(L_dpe_index,L_dpe_var,annee_en_cours):
    L_dpe_index_return=deepcopy(L_dpe_index)
    L_dpe_var_return=deepcopy(L_dpe_var)
    i=0
    while i<len(L_dpe_index_return):
        IPONDL=survie(L_dpe_var_return[i][0],L_dpe_index_return[i][0],annee_en_cours)
        if IPONDL==0:
            del L_dpe_index_return[i]
            del L_dpe_var_return[i]
        else:
            L_dpe_var_return[i][0]=IPONDL
        i+=1
    return L_dpe_index_return, L_dpe_var_return
    
## Rénovation
d_efficiency_max={'A':0,'B':0,'C':0.3,'D':0.3,'E':0.3,'F':0.3,'G':0.3}
d_efficiency_max_SNBC={'A':0,'B':0,'C':0.55,'D':0.55,'E':0.55,'F':0.55,'G':0.55}
year_ref=2020
year_optim=2030
def reno(energy_consumption,energy_class,year,SNBC=False):
    if SNBC:
        if year>=year_optim:
            return energy_consumption*(1-d_efficiency_max_SNBC[energy_class])
        else:
            return energy_consumption*(1-(d_efficiency_max[energy_class]+(year-year_ref)/(year_optim-year_ref)*(d_efficiency_max_SNBC[energy_class]-d_efficiency_max[energy_class])))
    else:
        return energy_consumption*(1-d_efficiency_max[energy_class])

def energy_to_class(energy):
    if energy<=50:
        return "A"
    elif energy<=90:
        return "B"
    elif energy<=150:
        return "C"
    elif energy<=230:
        return "D"
    elif energy<=330:
        return "E"
    elif energy<=450:
        return "F"
    else:
        return "G"

nbr_2020=400000
nbr_2030=680000
nbr_2040=830000
nbr_2050=830000
nbr_2060=830000
def nbr_reno(year):
    if year<=2030:
        return nbr_2020+(year-2020)/10*(nbr_2030-nbr_2020)
    elif year<=2040:
        return nbr_2030+(year-2030)/10*(nbr_2040-nbr_2030)
    elif year<=2050:
        return nbr_2040+(year-2040)/10*(nbr_2050-nbr_2040)
    else:
        return nbr_2050+(year-2050)/10*(nbr_2060-nbr_2050)

d_letter={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
L_letter=['A','B','C','D','E','F','G']
def renovation_global(L_dpe_index,L_dpe_var,year,SNBC=False):
    d_nbr_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
    for i in range(len(L_dpe_index)):
        d_nbr_class[L_dpe_index[i][-1]]+=L_dpe_var[i][0]
    
    p_reno_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
    N_reno=nbr_reno(year)
    L_cumul_nbr_class=[0 for i in range(7)]
    for i in range(7):
        L_cumul_nbr_class[i]=L_cumul_nbr_class[i-1]+d_nbr_class[L_letter[-i-1]]
    i_stop=0
    b_continue=True
    while b_continue and i_stop<7:
        if N_reno>=L_cumul_nbr_class[i_stop]:
            i_stop+=1
        else:
            b_continue=False
    L_cumul_nbr_class=[0]+L_cumul_nbr_class
    for i in range(i_stop):
        p_reno_class[L_letter[-i-1]]=1
    p_reno_class[L_letter[-i_stop-1]]=(N_reno-L_cumul_nbr_class[i_stop])/d_nbr_class[L_letter[-i_stop-1]]
    
    L_dpe_index_return=deepcopy(L_dpe_index)
    L_dpe_var_return=deepcopy(L_dpe_var)
    
    for i in range(len(L_dpe_index)):
        N_renov=round(L_dpe_var_return[i][0]*p_reno_class[L_dpe_index[i][-1]])
        if N_renov>0:
            L_dpe_var_return[i][0]-=N_renov
            new_energy=reno(L_dpe_var_return[i][1],L_dpe_index[i][-1],year,SNBC)
            new_class=energy_to_class(new_energy)
            New_index=L_dpe_index[i][0:-1]+[new_class]
            try:
                i_new=L_dpe_index_return.index(New_index)
                IPONDL_rec=L_dpe_var_return[i_new][0]
                energy_rec=L_dpe_var_return[i_new][1]
                energy_mean=(IPONDL_rec*energy_rec+N_renov*new_energy)/(IPONDL_rec+N_renov)
                L_dpe_var_return[i_new][0]+=N_renov
                L_dpe_var_return[i_new][1]=energy_mean
            except:
                L_dpe_index_return.append(New_index)
                L_dpe_var_return.append([N_renov,new_energy,L_dpe_var[i][-1]])
    return L_dpe_index_return,L_dpe_var_return

## Changement mode de chauffage
year_ref_trans=2020
year_optim_trans=2030
year_goal_trans=2050

year_optim_fioul=2024
year_goal_fioul=2030# Le gouvernement viserait la sortie complète du fioul en 2028 (PPE). 
#En plus cela aide à tenir l'objectif fit-for-55.

def mat_trans(L_dpe_index,L_dpe_var,year):
    P_trans_house=[[0 for i in range(8)] for j in range(8)]
    P_trans_apartment=[[0 for i in range(8)] for j in range(8)]
    
    L_heat_house_t=[0 for i in range(8)]
    L_heat_apartment_t=[0 for i in range(8)]
    for i in range(len(L_dpe_index)):
        if L_dpe_index[i][3]=="house":
            L_heat_house_t[d_heat[L_dpe_index[i][1]]]+=L_dpe_var[i][0]
        else:
            L_heat_apartment_t[d_heat[L_dpe_index[i][1]]]+=L_dpe_var[i][0]
    if year<=year_optim_fioul:
        for i in [0,2,3,4,5,6,7]:
            for j in range(8):
                #if j!=i:
                P_trans_house[i][j]=N_trans_house[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
                P_trans_apartment[i][j]=N_trans_apartment[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
        for j in range(8):
            #if j!=1:
            P_trans_house[1][j]=N_trans_house[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)\
                *(year-year_ref_trans)/(year_optim_fioul-year_ref_trans)
            P_trans_apartment[1][j]=N_trans_apartment[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)\
                *(year-year_ref_trans)/(year_optim_fioul-year_ref_trans)
    elif year<=year_optim_trans:
        for i in [0,2,3,4,5,6,7]:
            for j in range(8):
                #if j!=i:
                P_trans_house[i][j]=N_trans_house[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
                P_trans_apartment[i][j]=N_trans_apartment[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
        for j in range(8):
            #if j!=1:
            P_trans_house[1][j]=N_trans_house[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
            P_trans_apartment[1][j]=N_trans_apartment[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
    else:
        for i in [0,2,3,4,5,6,7]:
            for j in range(8):
                #if j!=i:
                P_trans_house[i][j]=N_trans_house[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)
                P_trans_apartment[i][j]=N_trans_apartment[i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)
        for j in range(8):
            #if j!=1:
            P_trans_house[1][j]=N_trans_house[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
            P_trans_apartment[1][j]=N_trans_apartment[1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
                
    for i in range(8):
        denom_house=max(L_heat_house_t[i],sum(P_trans_house[i]),1)        
        denom_apartment=max(L_heat_apartment_t[i],sum(P_trans_apartment[i]),1)
        for j in range(8):
            P_trans_house[i][j]=P_trans_house[i][j]/denom_house
            P_trans_apartment[i][j]=P_trans_apartment[i][j]/denom_apartment
            
    return P_trans_house,P_trans_apartment

def heat_trans(L_dpe_index,L_dpe_var,year):
    L_dpe_index_return=deepcopy(L_dpe_index)
    L_dpe_var_return=deepcopy(L_dpe_var)
    
    P_trans_house,P_trans_apartment=mat_trans(L_dpe_index,L_dpe_var,year)
    
    for i in range(len(L_dpe_index)):
        if L_dpe_index[i][3]=="house":
            L_trans=P_trans_house[d_heat[L_dpe_index[i][1]]]
        else:
            L_trans=P_trans_apartment[d_heat[L_dpe_index[i][1]]]
        N_trans=[]
        for j in range(8):
            if j!=d_heat[L_dpe_index[i][1]]:
                N_trans.append(round(L_dpe_var_return[i][0]*L_trans[j]))
            else:
                N_trans.append(0)
        while sum(N_trans)>L_dpe_var_return[i][0]:
            # Il arrive que l'on transitionne plus de logements qu'il y en a...
            j_max=N_trans.index(max(N_trans))
            N_trans[j_max]-=1
        
        for j in range(8):
            if N_trans[j]>0:
                New_index=[L_dpe_index[i][0],L_heat[j]]+L_dpe_index[i][2::]
                try:
                    i_new=L_dpe_index_return.index(New_index)
                    IPONDL_rec=L_dpe_var_return[i_new][0]
                    energy_rec=L_dpe_var_return[i_new][1]
                    energy_i=L_dpe_var_return[i][1]
                    energy_mean=(IPONDL_rec*energy_rec+N_trans[j]*energy_i)/(IPONDL_rec+N_trans[j])
                    L_dpe_var_return[i_new][0]+=N_trans[j]
                    L_dpe_var_return[i_new][1]=energy_mean
                except:
                    L_dpe_index_return.append(New_index)
                    L_dpe_var_return.append([N_trans[j]]+L_dpe_var_return[i][1::])
        L_dpe_var_return[i][0]-=sum(N_trans)
        
    
    return L_dpe_index_return,L_dpe_var_return
                
## Neuf
L_heat_new_house0=[0.18,0,0.24,0,0.03,0.41,0.14,0]
L_heat_new_house=[0.25,0,0,0,0.02,0.48,0.15,0.1]
L_heat_new_apartment0=[0,0,0.78,0.09,0.08,0.03,0.02,0]
L_heat_new_apartment=[0,0,0,0.2,0.05,0.35,0.35,0.05]

year_new_house=2024#2022 normalement mais les permis délivrés avant sont valables jusqu'à fin 2023
year_new_apartment=2026#Jusqu'à fin 2024 dans les logements collectifs il est possible d'installer du gaz
# Une marge pour le permis de construire est considérée aussi.

N_new_year=366700
energy_new=50

taux_house0=0.45
taux_housef=0.4
year_housef=2030

def taux_house(year):
    return taux_house0+(year-annee_ref)/(year_housef-annee_ref)*(taux_housef-taux_house0)

def add_new(L_dpe_index,L_dpe_var,year):
    L_dpe_index_return=deepcopy(L_dpe_index)
    L_dpe_var_return=deepcopy(L_dpe_var)
    
    N_new_house=round(taux_house(year)*N_new_year)
    N_new_apartment=N_new_year-N_new_house
    
    #House
    for i in range(8):#Heating
        for j in range(7):#Surface
            for k in range(2):#Owner or renter
                New_index=["[2013, 2100]",L_heat[i],L_surfaces[j],"house",L_status[k],"A"]
                if year>=year_new_house:
                    IPONDL=round(L_heat_new_house[i]*L_taux_new_house[j][k]*N_new_house)
                else:
                    IPONDL=round(L_heat_new_house0[i]*L_taux_new_house[j][k]*N_new_house)
                if IPONDL>0:
                    try:
                        i_new=L_dpe_index_return.index(New_index)
                        IPONDL_rec=L_dpe_var_return[i_new][0]
                        energy_rec=L_dpe_var_return[i_new][1]
                        energy_mean=(IPONDL_rec*energy_rec+IPONDL*energy_new)/(IPONDL_rec+IPONDL)
                        L_dpe_var_return[i_new][0]+=IPONDL
                        L_dpe_var_return[i_new][1]=energy_mean
                    except:
                        L_dpe_index_return.append(New_index)
                        L_dpe_var_return.append([IPONDL,energy_new,L_surf_house[j]])
    
    #Apartment
    for i in range(8):#Heating
        for j in range(7):#Surface
            for k in range(2):#Owner or renter
                New_index=["[2013, 2100]",L_heat[i],L_surfaces[j],"apartment",L_status[k],"A"]
                if year>=year_new_apartment:
                    IPONDL=round(L_heat_new_apartment[i]*L_taux_new_apartment[j][k]*N_new_apartment)
                else:
                    IPONDL=round(L_heat_new_apartment0[i]*L_taux_new_apartment[j][k]*N_new_apartment)
                if IPONDL>0:
                    try:
                        i_new=L_dpe_index_return.index(New_index)
                        IPONDL_rec=L_dpe_var_return[i_new][0]
                        energy_rec=L_dpe_var_return[i_new][1]
                        energy_mean=(IPONDL_rec*energy_rec+IPONDL*energy_new)/(IPONDL_rec+IPONDL)
                        L_dpe_var_return[i_new][0]+=IPONDL
                        L_dpe_var_return[i_new][1]=energy_mean
                    except:
                        L_dpe_index_return.append(New_index)
                        L_dpe_var_return.append([IPONDL,energy_new,L_surf_house[j]])
    return L_dpe_index_return,L_dpe_var_return


In [17]:
# Tests
t1=time()
L_dpe_index_test, L_dpe_var_test=survie_global(L_dpe_index, L_dpe_var, 2021)
L_dpe_index_return0,L_dpe_var_return0=renovation_global(L_dpe_index_test, L_dpe_var_test, 2021)
L_dpe_index_return1,L_dpe_var_return1=heat_trans(L_dpe_index_return0, L_dpe_var_return0, 2021)
L_dpe_index_return,L_dpe_var_return=add_new(L_dpe_index_return1, L_dpe_var_return1, 2021)
t2=time()
print(t2-t1)
d_nbr_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
for i in range(len(L_dpe_index_return)):
    d_nbr_class[L_dpe_index_return[i][-1]]+=L_dpe_var_return[i][0]
print(d_nbr_class)
L_heat_h=[0 for i in range(8)]
L_heat_a=[0 for i in range(8)]
for i in range(len(L_dpe_index_return)):
    if L_dpe_index_return[i][3]=="house":
        L_heat_h[d_heat[L_dpe_index_return[i][1]]]+=L_dpe_var_return[i][0]
    else:
        L_heat_a[d_heat[L_dpe_index_return[i][1]]]+=L_dpe_var_return[i][0]
print(L_heat_h)
print(L_heat_a)

2.7351322174072266
{'A': 1604385, 'B': 3876293, 'C': 8360964, 'D': 8210708, 'E': 4668045, 'F': 2000063, 'G': 231307}
[1234990, 3112932, 5452255, 13246, 5399150, 1099282, 42452, 3191]
[0, 457698, 5766179, 1566851, 3989123, 802731, 10831, 854]


In [18]:
## Simulation complète de la rénovation (référence)
#E_non_heat=18.8#en kWh/m2/an, moyenne ECS+clim 2018-2020
d_corr_class={"A":1.3,"B":1.1,"C":0.85,"D":0.75,"E":0.65,"F":0.6,"G":0.55}
N_heat_h=[]
E_heat_h=[]
N_class_h=[]
N_heat_a=[]
E_heat_a=[]
N_class_a=[]
L_dpe_indexi,L_dpe_vari=deepcopy(L_dpe_index),deepcopy(L_dpe_var)

# Etat 2020
N_heat_h.append([0 for i in range(8)])
E_heat_h.append([0 for i in range(8)])
N_class_h.append([0 for i in range(7)])
N_heat_a.append([0 for i in range(8)])
E_heat_a.append([0 for i in range(8)])
N_class_a.append([0 for i in range(7)])
for i in range(len(L_dpe_indexi)):
    j=d_heat[L_dpe_indexi[i][1]]
    if L_dpe_indexi[i][3]=="house":
        N_heat_h[-1][j]+=L_dpe_vari[i][0]
        E_heat_h[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
        #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
        N_class_h[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]
    else:
        N_heat_a[-1][j]+=L_dpe_vari[i][0]
        E_heat_a[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
        #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
        N_class_a[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]

# Evolution
for year in range(2021,2061):
    print(year)
    if year<=2050:
        L_dpe_indexi,L_dpe_vari=survie_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=renovation_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=heat_trans(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=add_new(L_dpe_indexi, L_dpe_vari, year)
    else:# Pas de changement de mix après 2050
        L_dpe_indexi,L_dpe_vari=survie_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=renovation_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=add_new(L_dpe_indexi, L_dpe_vari, year)
    N_heat_h.append([0 for i in range(8)])
    E_heat_h.append([0 for i in range(8)])
    N_class_h.append([0 for i in range(7)])
    N_heat_a.append([0 for i in range(8)])
    E_heat_a.append([0 for i in range(8)])
    N_class_a.append([0 for i in range(7)])
    for i in range(len(L_dpe_indexi)):
        j=d_heat[L_dpe_indexi[i][1]]
        if L_dpe_indexi[i][3]=="house":
            N_heat_h[-1][j]+=L_dpe_vari[i][0]
            E_heat_h[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
            #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
            N_class_h[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]
        else:
            N_heat_a[-1][j]+=L_dpe_vari[i][0]
            E_heat_a[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
            #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
            N_class_a[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]

filename="Nombre_maisons_type_de_chauffage_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+N_heat_h[year-2020])
filename="Nombre_appartements_type_de_chauffage_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+N_heat_a[year-2020])
filename="Energie_TWh_maisons_type_de_chauffage_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x/1e9 for x in E_heat_h[year-2020]])
filename="Energie_TWh_appartements_type_de_chauffage_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x/1e9 for x in E_heat_a[year-2020]])
filename="Nombre_maisons_classe_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_letter)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x for x in N_class_h[year-2020]])
filename="Nombre_appartements_classe_ref.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_letter)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x for x in N_class_a[year-2020]])

2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [19]:
## Simulation complète de la rénovation (SNBC)
#E_non_heat=18.8#en kWh/m2/an, moyenne ECS+clim 2018-2020
d_corr_class={"A":1.3,"B":1.1,"C":0.85,"D":0.75,"E":0.65,"F":0.6,"G":0.55}
N_heat_h=[]
E_heat_h=[]
N_class_h=[]
N_heat_a=[]
E_heat_a=[]
N_class_a=[]
L_dpe_indexi,L_dpe_vari=deepcopy(L_dpe_index),deepcopy(L_dpe_var)

# Etat 2020
N_heat_h.append([0 for i in range(8)])
E_heat_h.append([0 for i in range(8)])
N_class_h.append([0 for i in range(7)])
N_heat_a.append([0 for i in range(8)])
E_heat_a.append([0 for i in range(8)])
N_class_a.append([0 for i in range(7)])
for i in range(len(L_dpe_indexi)):
    j=d_heat[L_dpe_indexi[i][1]]
    if L_dpe_indexi[i][3]=="house":
        N_heat_h[-1][j]+=L_dpe_vari[i][0]
        E_heat_h[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
        #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
        N_class_h[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]
    else:
        N_heat_a[-1][j]+=L_dpe_vari[i][0]
        E_heat_a[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
        #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
        N_class_a[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]

# Evolution
for year in range(2021,2061):
    print(year)
    if year<=2050:
        L_dpe_indexi,L_dpe_vari=survie_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=renovation_global(L_dpe_indexi, L_dpe_vari, year,True)
        L_dpe_indexi,L_dpe_vari=heat_trans(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=add_new(L_dpe_indexi, L_dpe_vari, year)
    else:# Pas de changement de mix après 2050
        L_dpe_indexi,L_dpe_vari=survie_global(L_dpe_indexi, L_dpe_vari, year)
        L_dpe_indexi,L_dpe_vari=renovation_global(L_dpe_indexi, L_dpe_vari, year,True)
        L_dpe_indexi,L_dpe_vari=add_new(L_dpe_indexi, L_dpe_vari, year)
    N_heat_h.append([0 for i in range(8)])
    E_heat_h.append([0 for i in range(8)])
    N_class_h.append([0 for i in range(7)])
    N_heat_a.append([0 for i in range(8)])
    E_heat_a.append([0 for i in range(8)])
    N_class_a.append([0 for i in range(7)])
    for i in range(len(L_dpe_indexi)):
        j=d_heat[L_dpe_indexi[i][1]]
        if L_dpe_indexi[i][3]=="house":
            N_heat_h[-1][j]+=L_dpe_vari[i][0]
            E_heat_h[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
            #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
            N_class_h[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]
        else:
            N_heat_a[-1][j]+=L_dpe_vari[i][0]
            E_heat_a[-1][j]+=L_dpe_vari[i][0]*L_dpe_vari[i][1]*L_dpe_vari[i][2]*d_corr_class[L_dpe_indexi[i][-1]]*taux_heat
            #-E_non_heat*L_dpe_vari[i][2]*L_dpe_vari[i][0]
            N_class_a[-1][d_letter[L_dpe_indexi[i][5]]]+=L_dpe_vari[i][0]

filename="Nombre_maisons_type_de_chauffage_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+N_heat_h[year-2020])
filename="Nombre_appartements_type_de_chauffage_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+N_heat_a[year-2020])
filename="Energie_TWh_maisons_type_de_chauffage_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x/1e9 for x in E_heat_h[year-2020]])
filename="Energie_TWh_appartements_type_de_chauffage_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_heat)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x/1e9 for x in E_heat_a[year-2020]])
filename="Nombre_maisons_classe_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_letter)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x for x in N_class_h[year-2020]])
filename="Nombre_appartements_classe_SNBC.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_letter)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x for x in N_class_a[year-2020]])

2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
